In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import DatastreamDSWS as dsws
ds=dsws.Datastream(username='ben.mcdonald@omg.ca',password='Uuddlrlrba55')

In [113]:
daily_data = ds.get_data(tickers='MSACWFL(MSRI), MLUS710(RI), MLCORML(RI), MLHMACL(RI)',
                         fields=['PCH#(X,-1D)'], start='2002-01-01', end='0D', freq='D')
daily_data.index = pd.to_datetime(data.index)

In [115]:
monthly_data = ds.get_data(tickers='MSACWFL(MSRI), MLUS710(RI), MLCORML(RI), MLHMACL(RI), MSERCAD, MLCR1CL(RI)',
                           fields=['PCH#(X,-1M)'], start='2002-01-31', end='0D', freq='M')
monthly_data.index = pd.to_datetime(monthly_data.index)

In [116]:
MKT = daily_data.iloc[:,0]
INT = daily_data.iloc[:,1]
CRD = 0.5 * daily_data.iloc[:,2] + 0.5 * daily_data.iloc[:,3]

In [117]:
rolling_months = 36
month_end_dates = []
for d in monthly_data.index:
    
    while d not in daily_data.index:
        d - pd.Timedelta(days=1)
        
    month_end_dates.append(d)

In [118]:
wls = LinearRegression()
halflife = 260
betas = []

for i in range(len(month_end_dates)-rolling_months):
    
    start_date = month_end_dates[i]
    end_date = month_end_dates[i+rolling_months-1]
    rolling_days = data.loc[start_date : end_date].shape[0]
    
    beta = wls.fit(pd.concat([MKT, INT], axis=1).loc[start_date:end_date],
                   CRD.loc[start_date : end_date],
                   sample_weight = [(0.5 ** (1 / halflife)) ** i for i in range(rolling_days)][::-1]).coef_
    
    betas.append(beta)
    
betas = np.array(betas)

In [119]:
monthly_data = monthly_data.iloc[rolling_months:]
MKT = monthly_data.iloc[:,0]
INT = monthly_data.iloc[:,1]
CRD = 0.5 * monthly_data.iloc[:,2] + 0.5 * monthly_data.iloc[:,3]
CUR = monthly_data.iloc[:,4]
RF = monthly_data.iloc[:,5]

In [120]:
mkt_crd_betas = [b[0] for b in betas]
int_crd_betas = [b[1] for b in betas]

In [123]:
CRD = CRD - MKT * mkt_crd_betas - INT * int_crd_betas

In [130]:
monthly_data = pd.concat([MKT, INT, CRD, CUR, RF], axis=1)
monthly_data.columns = 'MKT INT CRD CUR RF'.split()

In [131]:
monthly_data

,MKT,INT,CRD,CUR,RF
Dates,,,,,
2005-01-31,-0.866,0.680,0.085720,3.55546,0.221
2005-02-28,2.787,-1.229,0.808200,-0.93088,0.198
2005-03-31,-1.174,-0.550,-1.584059,-1.57420,0.222
2005-04-29,-2.294,2.411,-0.845807,3.72775,0.207
2005-05-31,3.676,1.591,0.569927,-0.06376,0.226
...,...,...,...,...,...
2020-07-31,4.006,0.851,2.640098,-1.65198,0.045
2020-08-31,5.793,-0.835,-0.989691,-2.72867,0.043
2020-09-30,-2.701,0.266,-0.174976,2.51734,0.040
